In [3]:
import numpy as np
import pandas as pd
import keras
import imageio
import glob
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import SGD
from IPython.display import Markdown, display
import gc

def printmd(string):
    display(Markdown(string))

Загрузим датасет, разобьём на train 90%, val 10%, test 10%.

In [4]:
df = pd.read_csv('mnist.csv', dtype=np.float32)

train = df.sample(frac=0.8)
other = df.drop(train.index)
val   = other.sample(frac=0.5)
test  = other.drop(val.index)

split = lambda df: (
    df.drop('label', axis=1).as_matrix() / 255,            # 255 — map colors to [0..1]
    keras.utils.to_categorical(df.label.as_matrix(), 10))  # 10 — number of classes

trainX, trainY = split(train)
valX, valY     = split(val)
testX, testY   = split(test)

Функция будет обучать модель и смотреть точность.

In [16]:
def test(layers, test=True, print=True):
    keras.backend.clear_session()
    
    m = Sequential()
    for l in layers: m.add(l)
    
    m.compile(SGD(0.2), 'categorical_crossentropy', ['accuracy'])
    m.fit(trainX, trainY, batch_size=128, epochs=10, verbose=True, validation_data=(valX, valY))

    x, y = (testX, testY) if test else (valX, valY)
    loss, accu = m.evaluate(x, y, verbose=False)
    keras.backend.clear_session()
    
    title = '**test**' if test else '**val**'
    if print: printmd(f'{title}: loss {loss:.3f}, accuracy {accu:.3f}')
    return accu

### task 1

Обучите softmax регрессию на подготовленном датасете mnist.

In [46]:
test([
    Dense(10, activation='softmax', input_shape=(28 * 28,)),
])

**test**:  loss 0.305,   accuracy 0.903

### task 2

Добавьте один слой из 1024 нейронов с tanh функцией активации.

In [48]:
test([
    Dense(1024, activation='tanh', input_shape=(28 * 28,)),
    Dense(10,   activation='softmax'),
])

**test**:  loss 0.269,   accuracy 0.918

### task 3

Добавьте к нейронной сети слой из 32-х сверток с ядром 5x5 и шагом 2 с функцией активации ReLU.

In [49]:
test([
    Reshape((28, 28, 1), input_shape=(28 * 28,)),
    Conv2D(32, (5, 5), strides=2, activation='relu'),
    Flatten(),
    Dense(1024, activation='tanh'),
    Dense(10,   activation='softmax'),
])

**test**:  loss 0.109,   accuracy 0.969

### task 4

Добавьте к нейронной сети слой max pool ядром (2x2) и шагом 1.

In [50]:
test([
    Reshape((28, 28, 1), input_shape=(28 * 28,)),
    Conv2D(32, (5, 5), strides=2, activation='relu'),
    MaxPooling2D((2, 2), strides=1),
    Flatten(),
    Dense(1024, activation='tanh'),
    Dense(10,   activation='softmax'),
])

**test**:  loss 0.094,   accuracy 0.976

### task 5

Добавьте к нейронной сети регуляризацию dropout и оптимизируйте коэффициент, основываясь на validate сете.

In [15]:
run = lambda rate: test([
        Reshape((28, 28, 1), input_shape=(28 * 28,)),
        Conv2D(32, (5, 5), strides=2, activation='relu'),
        MaxPooling2D((2, 2), strides=1),
        Dropout(rate),
        Flatten(),
        Dense(1024, activation='tanh'),
        Dense(10,   activation='softmax'),
    ], test=False, print=False)

rates = np.linspace(0, 1, 20)
accus = np.array([run(rate) for rate in rates])

rate, acc = rates[accus.argmax()], accus.max()
print(f'best rate: {rate:.3f}, acc: {acc:.3f}')

best rate: 0.368, acc: 0.986


### task 6

Добавьте еще один слой свертки и еще один слой maxpool (параметры задайте самостоятельно).

In [35]:
test([
    Reshape((28, 28, 1), input_shape=(28 * 28,)),
    Conv2D(32, (5, 5), strides=2, activation='relu'),
    MaxPooling2D((2, 2), strides=1),
    Dropout(0.368),
    Conv2D(64, (5, 5), strides=2, activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Dropout(0.368),
    Flatten(),
    Dense(1024, activation='tanh'),
    Dense(10,   activation='softmax'),
])

**test**: loss 0.047, accuracy 0.988

0.988

### task 7

Подготовьте датасет notMNIST (выделите train/validate/test) и примените сетки из основного задания.

In [5]:
df = pd.read_csv('not_mnist.csv', dtype=np.float32)

train = df.sample(frac=0.8)
other = df.drop(train.index)
val   = other.sample(frac=0.5)
test  = other.drop(val.index)

split = lambda df: (
    df.drop('label', axis=1).as_matrix() / 255,            # 255 — map colors to [0..1]
    keras.utils.to_categorical(df.label.as_matrix(), 10))  # 10 — number of classes

trainX, trainY = split(train)
valX, valY     = split(val)
testX, testY   = split(test)

##### 1 sotmax

In [17]:
test([
    Dense(10, activation='softmax', input_shape=(28 * 28,)),
]) and None

Train on 14979 samples, validate on 1872 samples
Epoch 1/10
14979/14979 [==============================] - 1s 73us/step - loss: 0.7820 - acc: 0.8005 - val_loss: 0.5011 - val_acc: 0.8670
Epoch 2/10
14979/14979 [==============================] - 1s 60us/step - loss: 0.4461 - acc: 0.8811 - val_loss: 9.3912 - val_acc: 0.1239
Epoch 3/10
14979/14979 [==============================] - 1s 52us/step - loss: 0.5575 - acc: 0.8767 - val_loss: 0.6040 - val_acc: 0.8392
Epoch 4/10
14979/14979 [==============================] - 1s 52us/step - loss: 0.4082 - acc: 0.8911 - val_loss: 0.4662 - val_acc: 0.8846
Epoch 5/10
14979/14979 [==============================] - 1s 60us/step - loss: 0.3932 - acc: 0.8963 - val_loss: 0.4781 - val_acc: 0.8552
Epoch 6/10
14979/14979 [==============================] - 1s 41us/step - loss: 0.3867 - acc: 0.8972 - val_loss: 0.4175 - val_acc: 0.8916
Epoch 7/10
14979/14979 [==============================] - 1s 41us/step - loss: 0.3778 - acc: 0.8993 - val_loss: 0.5636 - val_acc:

**test**: loss 0.419, accuracy 0.893

###### 2 tanh, 1024

In [18]:
test([
    Dense(1024, activation='tanh', input_shape=(28 * 28,)),
    Dense(10,   activation='softmax'),
]) and None

Train on 14979 samples, validate on 1872 samples
Epoch 1/10
14979/14979 [==============================] - 1s 82us/step - loss: 0.9664 - acc: 0.8009 - val_loss: 0.4329 - val_acc: 0.8819
Epoch 2/10
14979/14979 [==============================] - 1s 55us/step - loss: 0.4058 - acc: 0.8904 - val_loss: 0.4181 - val_acc: 0.8884
Epoch 3/10
14979/14979 [==============================] - 1s 54us/step - loss: 0.3788 - acc: 0.8978 - val_loss: 0.4081 - val_acc: 0.8937
Epoch 4/10
14979/14979 [==============================] - 1s 57us/step - loss: 0.3614 - acc: 0.9009 - val_loss: 2.1407 - val_acc: 0.5855
Epoch 5/10
14979/14979 [==============================] - 1s 53us/step - loss: 0.4033 - acc: 0.8963 - val_loss: 0.4134 - val_acc: 0.8921
Epoch 6/10
14979/14979 [==============================] - 1s 54us/step - loss: 0.3409 - acc: 0.9054 - val_loss: 0.5840 - val_acc: 0.8349
Epoch 7/10
14979/14979 [==============================] - 1s 54us/step - loss: 0.3121 - acc: 0.9131 - val_loss: 0.5124 - val_acc:

**test**: loss 6.034, accuracy 0.261

###### 3 RELU

In [19]:
test([
    Reshape((28, 28, 1), input_shape=(28 * 28,)),
    Conv2D(32, (5, 5), strides=2, activation='relu'),
    Flatten(),
    Dense(1024, activation='tanh'),
    Dense(10,   activation='softmax'),
]) and None

Train on 14979 samples, validate on 1872 samples
Epoch 1/10
14979/14979 [==============================] - 3s 224us/step - loss: 0.7929 - acc: 0.7705 - val_loss: 0.4362 - val_acc: 0.8777
Epoch 2/10
14979/14979 [==============================] - 3s 172us/step - loss: 0.3323 - acc: 0.9063 - val_loss: 2.6328 - val_acc: 0.5422
Epoch 3/10
14979/14979 [==============================] - 3s 195us/step - loss: 0.5065 - acc: 0.8740 - val_loss: 0.4066 - val_acc: 0.8889
Epoch 4/10
14979/14979 [==============================] - 3s 171us/step - loss: 0.2745 - acc: 0.9204 - val_loss: 0.3401 - val_acc: 0.9113
Epoch 5/10
14979/14979 [==============================] - 2s 164us/step - loss: 0.2355 - acc: 0.9334 - val_loss: 0.3233 - val_acc: 0.9119
Epoch 6/10
14979/14979 [==============================] - 2s 166us/step - loss: 0.2114 - acc: 0.9388 - val_loss: 0.3156 - val_acc: 0.9204
Epoch 7/10
14979/14979 [==============================] - 2s 164us/step - loss: 0.1995 - acc: 0.9425 - val_loss: 0.4563 - v

**test**: loss 0.343, accuracy 0.919

###### 4 maxpool

In [20]:
test([
    Reshape((28, 28, 1), input_shape=(28 * 28,)),
    Conv2D(32, (5, 5), strides=2, activation='relu'),
    MaxPooling2D((2, 2), strides=1),
    Flatten(),
    Dense(1024, activation='tanh'),
    Dense(10,   activation='softmax'),
]) and None

Train on 14979 samples, validate on 1872 samples
Epoch 1/10
14979/14979 [==============================] - 3s 207us/step - loss: 0.8731 - acc: 0.7547 - val_loss: 2.4390 - val_acc: 0.4808
Epoch 2/10
14979/14979 [==============================] - 2s 161us/step - loss: 0.7954 - acc: 0.8408 - val_loss: 0.5223 - val_acc: 0.8499
Epoch 3/10
14979/14979 [==============================] - 2s 163us/step - loss: 0.3430 - acc: 0.9043 - val_loss: 0.4250 - val_acc: 0.8835
Epoch 4/10
14979/14979 [==============================] - 2s 155us/step - loss: 0.2851 - acc: 0.9209 - val_loss: 0.3656 - val_acc: 0.9097
Epoch 5/10
14979/14979 [==============================] - 2s 157us/step - loss: 0.2535 - acc: 0.9285 - val_loss: 1.6183 - val_acc: 0.6715
Epoch 6/10
14979/14979 [==============================] - 2s 155us/step - loss: 0.2622 - acc: 0.9260 - val_loss: 0.3183 - val_acc: 0.9119
Epoch 7/10
14979/14979 [==============================] - 2s 158us/step - loss: 0.2121 - acc: 0.9394 - val_loss: 0.3042 - v

**test**: loss 0.330, accuracy 0.907

###### 5 dropout

In [21]:
run = lambda rate: test([
        Reshape((28, 28, 1), input_shape=(28 * 28,)),
        Conv2D(32, (5, 5), strides=2, activation='relu'),
        MaxPooling2D((2, 2), strides=1),
        Dropout(rate),
        Flatten(),
        Dense(1024, activation='tanh'),
        Dense(10,   activation='softmax'),
    ], test=False, print=False)

rates = np.linspace(0, 1, 20)
accus = np.array([run(rate) for rate in rates])

rate, acc = rates[accus.argmax()], accus.max()
print(f'best rate: {rate:.3f}, acc: {acc:.3f}')

Train on 14979 samples, validate on 1872 samples
Epoch 1/10
14979/14979 [==============================] - 3s 173us/step - loss: 0.8170 - acc: 0.7660 - val_loss: 9.5112 - val_acc: 0.1357
Epoch 2/10
14979/14979 [==============================] - 2s 152us/step - loss: 0.5589 - acc: 0.8637 - val_loss: 4.0134 - val_acc: 0.4252
Epoch 3/10
14979/14979 [==============================] - 2s 152us/step - loss: 0.4032 - acc: 0.8927 - val_loss: 0.4264 - val_acc: 0.8894
Epoch 4/10
14979/14979 [==============================] - 2s 150us/step - loss: 0.2910 - acc: 0.9192 - val_loss: 9.0682 - val_acc: 0.1870
Epoch 5/10
14979/14979 [==============================] - 2s 151us/step - loss: 0.4481 - acc: 0.8922 - val_loss: 0.3650 - val_acc: 0.9065
Epoch 6/10
14979/14979 [==============================] - 2s 161us/step - loss: 0.2841 - acc: 0.9210 - val_loss: 1.5726 - val_acc: 0.7527
Epoch 7/10
14979/14979 [==============================] - 2s 165us/step - loss: 0.2756 - acc: 0.9256 - val_loss: 0.3356 - v

14979/14979 [==============================] - 3s 192us/step - loss: 0.2440 - acc: 0.9315 - val_loss: 0.2949 - val_acc: 0.9311
Epoch 9/10
14979/14979 [==============================] - 3s 169us/step - loss: 0.2127 - acc: 0.9364 - val_loss: 0.5585 - val_acc: 0.8627
Epoch 10/10
14979/14979 [==============================] - 3s 169us/step - loss: 0.2089 - acc: 0.9374 - val_loss: 0.2790 - val_acc: 0.9284
Train on 14979 samples, validate on 1872 samples
Epoch 1/10
14979/14979 [==============================] - 3s 209us/step - loss: 0.8262 - acc: 0.7555 - val_loss: 0.4658 - val_acc: 0.8723
Epoch 2/10
14979/14979 [==============================] - 2s 165us/step - loss: 0.3769 - acc: 0.8953 - val_loss: 0.4355 - val_acc: 0.8755
Epoch 3/10
14979/14979 [==============================] - 2s 163us/step - loss: 0.3333 - acc: 0.9068 - val_loss: 4.9475 - val_acc: 0.3002
Epoch 4/10
14979/14979 [==============================] - 2s 164us/step - loss: 0.4224 - acc: 0.8870 - val_loss: 1.1810 - val_acc: 0.

14979/14979 [==============================] - 3s 167us/step - loss: 0.4375 - acc: 0.8825 - val_loss: 0.3579 - val_acc: 0.9060
Epoch 6/10
14979/14979 [==============================] - 2s 164us/step - loss: 0.3418 - acc: 0.9010 - val_loss: 0.3131 - val_acc: 0.9193
Epoch 7/10
14979/14979 [==============================] - 2s 165us/step - loss: 0.3230 - acc: 0.9061 - val_loss: 3.1682 - val_acc: 0.5379
Epoch 8/10
14979/14979 [==============================] - 2s 164us/step - loss: 0.3922 - acc: 0.8937 - val_loss: 0.3283 - val_acc: 0.9092
Epoch 9/10
14979/14979 [==============================] - 3s 167us/step - loss: 0.3252 - acc: 0.9080 - val_loss: 7.2831 - val_acc: 0.1918
Epoch 10/10
14979/14979 [==============================] - 2s 165us/step - loss: 0.4524 - acc: 0.8817 - val_loss: 1.1277 - val_acc: 0.8007
Train on 14979 samples, validate on 1872 samples
Epoch 1/10
14979/14979 [==============================] - 3s 189us/step - loss: 0.7637 - acc: 0.7683 - val_loss: 0.3978 - val_acc: 0.

14979/14979 [==============================] - 3s 174us/step - loss: 0.7823 - acc: 0.7750 - val_loss: 0.5134 - val_acc: 0.8462
Epoch 3/10
14979/14979 [==============================] - 3s 170us/step - loss: 0.6427 - acc: 0.8189 - val_loss: 1.0809 - val_acc: 0.7057
Epoch 4/10
14979/14979 [==============================] - 3s 179us/step - loss: 0.6661 - acc: 0.8083 - val_loss: 0.5179 - val_acc: 0.8429
Epoch 5/10
14979/14979 [==============================] - 3s 173us/step - loss: 0.5787 - acc: 0.8349 - val_loss: 0.3860 - val_acc: 0.8862
Epoch 6/10
14979/14979 [==============================] - 3s 182us/step - loss: 0.5349 - acc: 0.8483 - val_loss: 0.3762 - val_acc: 0.8969
Epoch 7/10
14979/14979 [==============================] - 3s 190us/step - loss: 0.5393 - acc: 0.8469 - val_loss: 0.3609 - val_acc: 0.9049
Epoch 8/10
14979/14979 [==============================] - 3s 174us/step - loss: 0.5130 - acc: 0.8540 - val_loss: 0.3742 - val_acc: 0.8974
Epoch 9/10
14979/14979 [=====================

###### 6 total

In [22]:
test([
    Reshape((28, 28, 1), input_shape=(28 * 28,)),
    Conv2D(32, (5, 5), strides=2, activation='relu'),
    MaxPooling2D((2, 2), strides=1),
    Dropout(rate),
    Flatten(),
    Dense(1024, activation='tanh'),
    Dense(10,   activation='softmax'),
]) and None

Train on 14979 samples, validate on 1872 samples
Epoch 1/10
14979/14979 [==============================] - 3s 208us/step - loss: 0.7463 - acc: 0.7742 - val_loss: 0.3881 - val_acc: 0.8905
Epoch 2/10
14979/14979 [==============================] - 3s 168us/step - loss: 0.3556 - acc: 0.9001 - val_loss: 0.9001 - val_acc: 0.7959
Epoch 3/10
14979/14979 [==============================] - 2s 165us/step - loss: 0.3168 - acc: 0.9112 - val_loss: 1.1609 - val_acc: 0.7997
Epoch 4/10
14979/14979 [==============================] - 2s 166us/step - loss: 0.2976 - acc: 0.9178 - val_loss: 1.8607 - val_acc: 0.6303
Epoch 5/10
14979/14979 [==============================] - 3s 169us/step - loss: 0.2873 - acc: 0.9202 - val_loss: 0.3072 - val_acc: 0.9215
Epoch 6/10
14979/14979 [==============================] - 3s 169us/step - loss: 0.2434 - acc: 0.9311 - val_loss: 0.2984 - val_acc: 0.9193
Epoch 7/10
14979/14979 [==============================] - 2s 165us/step - loss: 0.2295 - acc: 0.9323 - val_loss: 0.3837 - v

**test**: loss 0.270, accuracy 0.926

### task 8

Сравните время работы самой глубокой полученной сети на CPU и GPU.

In [7]:
run = lambda: test([
    Reshape((28, 28, 1), input_shape=(28 * 28,)),
    Conv2D(32, (5, 5), strides=2, activation='relu'),
    MaxPooling2D((2, 2), strides=1),
    Dropout(0.368),
    Conv2D(64, (5, 5), strides=2, activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Dropout(0.368),
    Flatten(),
    Dense(1024, activation='tanh'),
    Dense(10,   activation='softmax'),
])

In [38]:
%%time

print('GPU')
run()

GPU


**test**: loss 0.059, accuracy 0.985

CPU times: user 10.3 s, sys: 1.77 s, total: 12.1 s
Wall time: 10.4 s


In [10]:
# restart the kernel, run this cell, then the 1st and following

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [9]:
%%time

import tensorflow as tf
with tf.device('/cpu:0'):
    print('CPU')
    run()

CPU


**test**: loss 14.619, accuracy 0.093

CPU times: user 1min 3s, sys: 12.9 s, total: 1min 16s
Wall time: 26.2 s


<b style="color:#ff5722">Итог</b>: время на GPU: 12 секунд, время на CPU: 66 секунд.